## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
# Import Libraries
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# build the code to scrape the following Wikipedia page
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
toronto = requests.get(URL)
print(toronto.headers)

{'Date': 'Tue, 17 Dec 2019 01:46:03 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.5', 'X-ATS-Timestamp': '1576547163', 'Content-Type': 'text/html; charset=UTF-8', 'X-Powered-By': 'PHP/7.2.24-1+0~20191026.31+debian9~1.gbpbbacde+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Last-Modified': 'Fri, 13 Dec 2019 03:18:54 GMT', 'Backend-Timing': 'D=102301 t=1576207541686528', 'Content-Encoding': 'gzip', 'Content-Length': '15091', 'X-Varnish': '320282536 283110963', 'Age': '10034', 'X-Cache': 'cp2012 hit, cp2004 hit/10', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=17-Dec-2019;Path=/;HttpOnly;secure;Expires=Sat, 18 Jan 2020 00:00:00 GMT, WMF-Last-Access-Global=17-Dec-2019;Path=/;Domain=.wikipedia.org;HttpOnly;secure;

In [3]:
# Clean the data
soup = BeautifulSoup(toronto.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

In [4]:
# Convert into a dataframe
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
df.head()

(210, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [5]:
# Groupby postalcode
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [6]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# Check the information for M5A, but checking with wiki table, there's only one neighborhood with postal code 'M5A'
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,Harbourfront


In [8]:
# Try with M1C which with several neighborhood
df.loc[df['PostalCode'] == 'M1C']

,PostalCode,Borough,Neighborhood
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"


In [9]:
# Check the information for M7A with no neighborhood information
# According to the rule it should be the same with Borough "Queen's Park"
df.loc[df['PostalCode'] == 'M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [10]:
# Check df.shape
df.shape

(103, 3)

### Use Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood

In [11]:
import geocoder

In [12]:
# Copy the previous dataframe
torontodf = df.copy()
torontodf.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
# Read Location Coordinates
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Map the previous dataframe with coordinates information
toronto_data = pd.merge(torontodf, coordinates, left_on='PostalCode', right_on='Postal Code').drop(['Postal Code'], axis = 1)
toronto_data.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
# Save Data
toronto_data.to_csv('Toronto_Data_with_coordinates.csv', index = False)

In [16]:
# Check the shape of data 
toronto_data.shape

(103, 5)

### Explore and cluster the neighborhoods in Toronto

In [51]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

from tqdm import tqdm
from collections import deque
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [18]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Torontousing latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
CLIENT_ID = 'LZ24YGJYBC1VJ1ZSBMZPN3E5TCAGQFRY0WHAI12N0DQF01CC' # your Foursquare ID
CLIENT_SECRET = 'WXQIFEIODVLPZWSJXU0FADRVQ4JO5HTCPFORHSA1AUXQ53JU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LZ24YGJYBC1VJ1ZSBMZPN3E5TCAGQFRY0WHAI12N0DQF01CC
CLIENT_SECRET:WXQIFEIODVLPZWSJXU0FADRVQ4JO5HTCPFORHSA1AUXQ53JU


In [29]:
#Define a function that takes the names and locations of the neighborhoods in Toronto and obtains the 100 top venues around.
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
Toronto_venues = getNearbyVenues(toronto_data.Neighborhood,
                            toronto_data.Latitude,
                            toronto_data.Longitude)

In [33]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


In [46]:
#take the venue category information and create a dataframe with a one hot enconding of these data.
Toronto_new = pd.get_dummies(Toronto_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_new["Neighborhood"] = Toronto_venues["Neighborhood"]


nindex = list(Toronto_new.columns).index("Neighborhood")
cols = deque(Toronto_new.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_new = Toronto_new[cols]

Toronto_new.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,...,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [47]:
# Get the average number of venue categories per neighborhood
Toronto_grouped = Toronto_new.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,...,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue
0,"Adelaide, King, Richmond",0.01,0.0,0.01,0.01,0.01,0.0,0.0,0.0,0.010000,...,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.333333,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [48]:
# define a function to chech for the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
# get the top 10 venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Asian Restaurant,Bar,Salad Place,Bakery,Burger Joint,Restaurant
1,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Clothing Store,Argentinian Restaurant,Airport Service,Airport Terminal,American Restaurant,Antique Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Arts & Crafts Store,Airport Service,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Music Venue
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pharmacy,Fried Chicken Joint,Video Store,Sandwich Place,Grocery Store,Pizza Place,Fast Food Restaurant,Beer Store,Music Venue,American Restaurant
4,"Alderwood, Long Branch",Pizza Place,Gym,Dance Studio,Pub,Pharmacy,Coffee Shop,Sandwich Place,Skating Rink,BBQ Joint,Auto Workshop


### Clustering

In [52]:
# use PCA decomposition to reduce the noise
pca = PCA(.95)
Toronto_grouped_clustering = pca.fit_transform(Toronto_grouped.drop('Neighborhood', 1))
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

In [53]:
# Kmeans clustering

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[4 4 2 1 1 1 4 4 4 4]
(101,)


In [56]:
# create a new dataframe with all the geo information along with the top 10 venues
Toronto_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
Toronto_combined = toronto_data.merge(Toronto_grouped, left_on = "Neighborhood", right_on = "Neighborhood", how = "outer")

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_combined = Toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_combined["Cluster Labels"] = Toronto_combined["Cluster Labels"].fillna(5).astype("int")

Toronto_combined.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,New American Restaurant,Nightclub,Noodle House,Office,Opera House,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,0.0,0.0,0.0,0.0,...,Fast Food Restaurant,Print Shop,Music Venue,Art Gallery,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Arts & Crafts Store,Bakery
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,0.0,0.0,0.0,0.0,...,Moving Target,Bar,Construction & Landscaping,Music Venue,Art Gallery,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,0.0,0.0,0.0,0.0,...,Electronics Store,Pizza Place,Breakfast Spot,Intersection,Medical Center,Spa,Rental Car Location,Mexican Restaurant,Argentinian Restaurant,American Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,0.0,0.0,0.0,0.0,...,Coffee Shop,Pharmacy,Korean Restaurant,Art Gallery,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Music Venue
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,0.0,0.0,0.0,0.0,...,Caribbean Restaurant,Gas Station,Bakery,Thai Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Hakka Restaurant,Aquarium,Airport Service


In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_combined['Latitude'],
                                  Toronto_combined['Longitude'],
                                  Toronto_combined['Neighborhood'],
                                  Toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters